In [1]:
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt 
import s3fs
import warnings 
import math

Open U data

In [ ]:
# open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path = 's3://velocity-u.zarr/'
remote_files = fsg.glob(data_path)

store = s3fs.S3Map(root=data_path, s3=fsg, check=False)
du = xr.open_zarr(store=store, consolidated=True)
du = du.rename({'depthu' : 'depth'})
du 

Open V data 

In [ ]:
 open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path = 's3://velocity-v.zarr/'
remote_files = fsg.glob(data_path)

store = s3fs.S3Map(root=data_path, s3=fsg, check=False)
dv = xr.open_zarr(store=store, consolidated=True)
dv = dv.rename({'depthv' : 'depth'})
dv 

In [ ]:
u = du.vozocrtx
v = dv.vomectry

Ubar and Vbar

In [ ]:
ubar = u[0,:,:,:]
vbar = v[0,:,:,:]
N = 0

for i in range(1,len(u.time_counter)):
    print(i) 
    ubar += u[i,:,:,:]
    vbar += v[i,:,:,:]
    N += 1
    
ubar = ubar/N
vbar = vbar/N

Uvar and Vvar

In [ ]:
uvar = (u[0,:,:,:])**2
vvar = (v[0,:,:,:])**2
N = 0

for i in range(1,len(u.time_counter)):
    print(i) 
    ubar += (u[i,:,:,:])**2
    vbar += (v[i,:,:,:])**2
    N += 1
    
uvar = uvar/N - ubar**2
vvar = vvar/N - vbar**2

UVbar

In [ ]:
uvbar = (u[0,:,:,:]*v[0,:,:,:])
N = 0 

for i in range(1,len(u.time_counter)):
    print(i) 
    uvbar += (u[i,:,:,:]*v[i,:,:,:])
    N += 1
uvbar = uvbar/N - ubar*vbar 

Merge to one dataset 

In [ ]:
covmatrix_data = xr.merge([ubar.to_dataset(name = 'ubar'),
                         vbar.to_dataset(name = 'vbar'),
                         uvar.to_dataset(name = 'uvar'),
                         vvar.to_dataset(name = 'vvar'),
                         uvbar.to_dataset(name = 'uvbar')], compat='override')